In [1]:
import pandas as pd
import numpy as np
from scipy.io import arff
import pprint

import os
import sys
from pathlib import Path

from sklearn.model_selection import train_test_split

project_root = Path().resolve().parents[0]
sys.path.append(str(project_root / "src"))

## load in a ton of code
from paths import CLEAN_DATA_DIR, MODELS_DIR, SEED
from utils import *
from models import *
from viz import *

In [ ]:
data = read_pickle(CLEAN_DATA_DIR, 'modeling_data.pkl')
ytrain = data['yTrain'] 
ytest = data['ytest'] 
Xtrain = data['Xtrain']
Xtest = data['Xtest']

In [3]:
lasso_results = fit_lasso_cv(Xtrain, ytrain, seed=SEED)

In [4]:
#best_rf = fit_rf_cv(Xtrain, ytrain, seed=SEED)

In [5]:
#best_xgb = fit_xgb_cv(Xtrain, ytrain, seed=SEED)

Now let's write all of these to pickle files for safe keeping

In [ ]:
#write_pickle(MODELS_DIR, "lasso_results.pkl", lasso_results)
#write_pickle(MODELS_DIR, "best_rf.pkl", lasso_results)
#write_pickle(MODELS_DIR, "best_xgb.pkl", lasso_results)

In [ ]:
lasso_results.keys()
print(len(lasso_results['Cs']))
print(len(lasso_results['coefs'][0]))

In [ ]:
plot_lasso_paths(lasso_results)